In [52]:
import requests
import json

file = open('API_KEY.txt', 'r')
API_KEY = file.read()

SPORT = 'upcoming' # use the sport_key from the /sports endpoint below, or use 'upcoming' to see the next 8 games across all sports

REGIONS = 'us' # uk | us | eu | au. Multiple can be specified if comma delimited

MARKETS = 'h2h' # h2h | spreads | totals. Multiple can be specified if comma delimited

ODDS_FORMAT = 'decimal' # decimal | american

DATE_FORMAT = 'iso' # iso | unix

odds_response = requests.get(
    f'https://api.the-odds-api.com/v4/sports/{SPORT}/odds',
    params={
        'api_key': API_KEY,
        'regions': REGIONS,
        'markets': MARKETS,
        'oddsFormat': ODDS_FORMAT,
        'dateFormat': DATE_FORMAT,
    }
)

if odds_response.status_code != 200:
    print(f'Failed to get odds: status_code {odds_response.status_code}, response body {odds_response.text}')

else:
    odds_json = odds_response.json()
    print('Number of events:', len(odds_json))
    print(json.dumps(odds_json, indent=4))

    # Check the usage quota
    print('Remaining requests', odds_response.headers['x-requests-remaining'])
    print('Used requests', odds_response.headers['x-requests-used'])

Number of events: 61
[
    {
        "id": "be957608430423e25d42887144c88c5b",
        "sport_key": "baseball_ncaa",
        "sport_title": "NCAA Baseball",
        "commence_time": "2024-04-14T16:00:00Z",
        "home_team": "Florida Int'l Golden Panthers",
        "away_team": "Wofford Terriers",
        "bookmakers": [
            {
                "key": "fanduel",
                "title": "FanDuel",
                "last_update": "2024-04-14T19:02:24Z",
                "markets": [
                    {
                        "key": "h2h",
                        "last_update": "2024-04-14T19:02:24Z",
                        "outcomes": [
                            {
                                "name": "Florida Int'l Golden Panthers",
                                "price": 4.5
                            },
                            {
                                "name": "Wofford Terriers",
                                "price": 1.18
                            }
 

In [57]:
# Event object == individual sporting event
class Event:
    def __init__(self, data):
        self.data = data
        self.sport_key = data['sport_key']
        self.id = data['id']

    def num_of_outcomes(self):
        bookmakers = self.data['bookmakers']
        total = []
        for bookmaker in bookmakers:
            odds = bookmaker['markets'][0]['outcomes']
            total.extend(odds)

        num_outcomes = len(total)
        return num_outcomes

    def pull_odds(self): #
        master_list = []

        bookmakers = self.data['bookmakers']
        for bookmaker in bookmakers:
            odds = bookmaker['markets'][0]['outcomes']

            for outcome in odds:

                team_name = outcome['name']
                odds_price = outcome['price']
                bookie = bookmaker['title']

                info = {
                    "team name": team_name,
                    "odds" : odds_price,
                    "bookmaker" : bookie,
                }
                
                master_list.append(info)
        return master_list
    
    def calculate_best_odds(self, list): # | list = master_list 

        outcome_1_name = None
        outcome_2_name = None
        outcome_3_name = None

        outcome_1_best_odds = float("-inf")
        outcome_2_best_odds = float("-inf")
        outcome_3_best_odds = float("-inf")

        outcome_1_bookmaker = None
        outcome_2_bookmaker = None
        outcome_3_bookmaker = None

        for outcome in range(len(list)):
            
            if outcome_1_name is None or outcome_1_name == list[outcome]['team name']:
                outcome_1_name = list[outcome]['team name']
                if list[outcome]['odds'] > outcome_1_best_odds:
                    outcome_1_best_odds = list[outcome]['odds']
                    outcome_1_bookmaker = list[outcome]['bookmaker']
            
            elif outcome_2_name is None or outcome_2_name == list[outcome]['team name']:
                outcome_2_name = list[outcome]['team name']
                if list[outcome]['odds'] > outcome_2_best_odds:
                    outcome_2_best_odds = list[outcome]['odds']
                    outcome_2_bookmaker = list[outcome]['bookmaker']
            
            if outcome_3_name is None and outcome_1_name != list[outcome]['team name'] and outcome_2_name != list[outcome]['team name']:
                outcome_3_name = list[outcome]['team name']

            if outcome_3_name == list[outcome]['team name']:
                if list[outcome]['odds'] > outcome_3_best_odds:
                    outcome_3_best_odds = list[outcome]['odds']
                    outcome_3_bookmaker = list[outcome]['bookmaker']

        return [[outcome_1_name, outcome_1_best_odds, outcome_1_bookmaker], [outcome_2_name, outcome_2_best_odds, outcome_2_bookmaker], [outcome_3_name, outcome_3_best_odds, outcome_3_bookmaker]]

        # Find best odds for each outcome of a matchup and return the [Team, Odds, Bookie] for each outcome

    # Calculate arbitrage | Boolean
    def calculate_arbitrage(self, best_odds_list):
        # If only 2 outcomes
        if best_odds_list[2][0] == None:
            best_odds_list.pop(2)
        
        return best_odds_list
    
    def american_to_decimal(self, american_odds):
        if american_odds > 0:
            return (american_odds / 100) + 1
        else:
            return (100 / american_odds) + 1
        
    def decimal_to_american(self, decimal_odds):
        if decimal_odds >= 2:
            return (decimal_odds - 1) * 100
        

    # Calculate arbitrage bet return | [Multiplier]

In [58]:
for i in range(len(odds_json)):
    game = Event(odds_json[i])
    master_list = game.pull_odds()
    num_outcomes = game.num_of_outcomes()
    odds = game.calculate_best_odds(master_list)

    print(master_list)
    print(num_outcomes)
    print(game.calculate_arbitrage(odds))
    print()

[{'team name': "Florida Int'l Golden Panthers", 'odds': 4.5, 'bookmaker': 'FanDuel'}, {'team name': 'Wofford Terriers', 'odds': 1.18, 'bookmaker': 'FanDuel'}, {'team name': "Florida Int'l Golden Panthers", 'odds': 5.5, 'bookmaker': 'MyBookie.ag'}, {'team name': 'Wofford Terriers', 'odds': 1.12, 'bookmaker': 'MyBookie.ag'}]
4
[["Florida Int'l Golden Panthers", 5.5, 'MyBookie.ag'], ['Wofford Terriers', 1.18, 'FanDuel']]

[{'team name': 'Boston College Eagles', 'odds': 10.5, 'bookmaker': 'MyBookie.ag'}, {'team name': 'Wake Forest Demon Deacons', 'odds': 1.02, 'bookmaker': 'MyBookie.ag'}, {'team name': 'Boston College Eagles', 'odds': 13.0, 'bookmaker': 'FanDuel'}, {'team name': 'Wake Forest Demon Deacons', 'odds': 1.02, 'bookmaker': 'FanDuel'}]
4
[['Boston College Eagles', 13.0, 'FanDuel'], ['Wake Forest Demon Deacons', 1.02, 'MyBookie.ag']]

[{'team name': 'Appalachian St Mountaineers', 'odds': 3.25, 'bookmaker': 'FanDuel'}, {'team name': 'Troy Trojans', 'odds': 1.31, 'bookmaker': 'FanDu